In [1]:
%pip install PyExifTool

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.2/51.2 kB 193.0 kB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
from ultralytics import YOLOv10
import supervision as sv

# Load YOLO model
model = YOLOv10("../yolos/yolov10/yolov10s_bs_16_extended/detect/train2/weights/best.pt")

# Load the dataset
dataset = sv.DetectionDataset.from_yolo(
    images_directory_path="../data/images/test",
    annotations_directory_path="../data/labels/test",
    data_yaml_path="../yolos/dataset.yaml"
)

/Users/dolphinium/anaconda3/envs/yolov10/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/Users/dolphinium/anaconda3/envs/yolov10/lib/python3.9/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c1017RegisterOperatorsD1Ev
  Referenced from: <2D1B8D5C-7891-3680-9CF9-F771AE880676> /Users/dolphinium/anaconda3/envs/yolov10/lib/python3.9/site-packages/torchvision/image.so
  Expected in:     <F9338F3C-C897-38D4-84EB-C2CFA21569E2> /Users/dolphinium/anaconda3/envs/yolov10/lib/python3.9/site-packages/torch/lib/libtorch_cpu.dylib'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
/Users/dolphinium/anaconda3/envs/yolov10/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please up

In [17]:
from exiftool import ExifToolHelper

tag_list = [
        "Composite:FOV",
        "Composite:GPSLatitude",
        "Composite:GPSLongitude",
        "XMP:AbsoluteAltitude",
        "XMP:RelativeAltitude",
        "XMP:GimbalRollDegree",
        "XMP:GimbalYawDegree",
        "XMP:GimbalPitchDegree"
        ]

image_path_list = dataset.images.keys()

with ExifToolHelper() as et:
    for image in image_path_list:
        print("metrics for image: ", image)
        for d in et.get_metadata(image):
            for k, v in d.items():
                if k in tag_list:
                    print(f"Dict: {k} = {v}")
        print("***************")            

SupervisionWarnings: images is deprecated: `DetectionDataset.images` property is deprecated and will be removed in `supervision-0.26.0`. Iterate with `for path, image, annotation in dataset:` instead.


metrics for image:  ../data/images/test/10849.jpg
Dict: XMP:AbsoluteAltitude = +47.81
Dict: XMP:RelativeAltitude = +60.90
Dict: XMP:GimbalRollDegree = +0.00
Dict: XMP:GimbalYawDegree = +30.80
Dict: XMP:GimbalPitchDegree = -89.9
Dict: Composite:GPSLatitude = 29.9522881944444
Dict: Composite:GPSLongitude = -85.4263130833333
Dict: Composite:FOV = 69.3903656740024
***************
metrics for image:  ../data/images/test/10801.jpg
Dict: XMP:AbsoluteAltitude = +48.01
Dict: XMP:RelativeAltitude = +61.10
Dict: XMP:GimbalRollDegree = +0.00
Dict: XMP:GimbalYawDegree = +31.00
Dict: XMP:GimbalPitchDegree = -89.9
Dict: Composite:GPSLatitude = 29.9515737222222
Dict: Composite:GPSLongitude = -85.4293835277778
Dict: Composite:FOV = 69.3903656740024
***************
metrics for image:  ../data/images/test/10807.jpg
Dict: XMP:AbsoluteAltitude = +47.81
Dict: XMP:RelativeAltitude = +60.90
Dict: XMP:GimbalRollDegree = +0.00
Dict: XMP:GimbalYawDegree = +31.00
Dict: XMP:GimbalPitchDegree = -89.9
Dict: Composit

In [4]:
import cv2
import numpy as np
import supervision as sv
from ultralytics import YOLOv10
from geopy.distance import geodesic
import folium
from exiftool import ExifToolHelper

# Constants
IMAGE_WIDTH = 4000
IMAGE_HEIGHT = 3000

# Load YOLO model
model = YOLOv10("../yolos/yolov10/yolov10s_bs_16_extended/detect/train2/weights/best.pt")

# Load the dataset
dataset = sv.DetectionDataset.from_yolo(
    images_directory_path="../data/images/test",
    annotations_directory_path="../data/labels/test",
    data_yaml_path="../yolos/dataset.yaml"
)

# Annotators
bounding_box_annotator = sv.BoxAnnotator()
label_annotator = sv.LabelAnnotator()

# Function to calculate ground distance from pixel distance
def calculate_ground_distance(altitude, fov_deg, image_dimension, pixel_distance):
    fov_rad = np.radians(fov_deg)
    ground_distance = (2 * altitude * np.tan(fov_rad / 2)) * (pixel_distance / image_dimension)
    return ground_distance

# Function to get GPS coordinates from offsets
def get_gps_coordinates(lat, lon, north_offset, east_offset):
    new_location = geodesic(meters=north_offset).destination((lat, lon), 0)
    new_location = geodesic(meters=east_offset).destination(new_location, 90)
    return new_location.latitude, new_location.longitude

# Extract metadata and process each image
building_locations = list()
tag_list = [
    "Composite:FOV",
    "Composite:GPSLatitude",
    "Composite:GPSLongitude",
    "XMP:AbsoluteAltitude",
    "XMP:RelativeAltitude",
    "XMP:GimbalRollDegree",
    "XMP:GimbalYawDegree",
    "XMP:GimbalPitchDegree"
]

with ExifToolHelper() as et:
    for image_path in dataset.images.keys():
        print(f"Processing image: {image_path}")
        
        # Extract metadata
        for d in et.get_metadata(image_path):
            metadata = {k: v for k, v in d.items() if k in tag_list}

        # Extract necessary metadata
        CAMERA_GPS = (metadata["Composite:GPSLatitude"], metadata["Composite:GPSLongitude"])
        ABSOLUTE_ALTITUDE = float(metadata["XMP:AbsoluteAltitude"])
        RELATIVE_ALTITUDE = float(metadata["XMP:RelativeAltitude"])
        GIMBAL_YAW_DEGREE = float(metadata["XMP:GimbalYawDegree"])
        GIMBAL_PITCH_DEGREE = float(metadata["XMP:GimbalPitchDegree"])
        FOV_HORIZONTAL = float(metadata["Composite:FOV"])
        FOV_VERTICAL = FOV_HORIZONTAL * (IMAGE_HEIGHT / IMAGE_WIDTH)  # Assuming square pixels
        
        # Convert degrees to radians
        yaw_rad = np.radians(GIMBAL_YAW_DEGREE)
        pitch_rad = np.radians(GIMBAL_PITCH_DEGREE)
        
        # Load image
        random_image = dataset.images[image_path]
        
        # Perform detection
        results = model(source=random_image, conf=0.25)[0]
        detections = sv.Detections.from_ultralytics(results)
        
        # Annotate image
        annotated_image = bounding_box_annotator.annotate(
            scene=random_image, detections=detections)
        annotated_image = label_annotator.annotate(
            scene=annotated_image, detections=detections)
        
        # Plot the annotated image (optional, for visual debugging)
        #sv.plot_image(annotated_image)
        
        # Process each detection
        for i, box in enumerate(detections.xyxy):
            x_min, y_min, x_max, y_max = box
            x_center = (x_min + x_max) / 2
            y_center = (y_min + y_max) / 2

            pixel_distance_x = x_center - IMAGE_WIDTH / 2
            pixel_distance_y = IMAGE_HEIGHT / 2 - y_center

            ground_distance_x = calculate_ground_distance(RELATIVE_ALTITUDE, FOV_HORIZONTAL, IMAGE_WIDTH, pixel_distance_x)
            ground_distance_y = calculate_ground_distance(RELATIVE_ALTITUDE, FOV_VERTICAL, IMAGE_HEIGHT, pixel_distance_y)

            east_offset = ground_distance_x * np.cos(yaw_rad) - ground_distance_y * np.sin(yaw_rad)
            north_offset = ground_distance_x * np.sin(yaw_rad) + ground_distance_y * np.cos(yaw_rad)

            building_lat, building_lon = get_gps_coordinates(CAMERA_GPS[0], CAMERA_GPS[1], north_offset, east_offset)
            building_locations.append((building_lat, building_lon, detections.class_id[i]))
            print(f"Building {i+1}: Latitude: {building_lat}, Longitude: {building_lon}")

# Create a Folium map centered at the first image's GPS position with satellite tiles
map_center = CAMERA_GPS
m = folium.Map(
    location=map_center,
    zoom_start=18,  # Increase zoom level for closer view
    tiles='Esri.WorldImagery'  # Use Esri's World Imagery for satellite view
)

# Add markers for each detected building
for i, (building_lat, building_lon, class_id) in enumerate(building_locations):
    building_status = 'Damaged' if class_id == 1 else 'Undamaged'
    
    folium.Marker(
        location=(building_lat, building_lon),
        popup=f'Building {i+1}: {building_status}',
        icon=folium.Icon(color='red' if building_status == 'Damaged' else 'green', icon='home')
    ).add_to(m)

# Save the map to an HTML file
m.save('detected_buildings_satellite_map.html')
m

SupervisionWarnings: images is deprecated: `DetectionDataset.images` property is deprecated and will be removed in `supervision-0.26.0`. Iterate with `for path, image, annotation in dataset:` instead.


Processing image: ../data/images/test/15885.jpg


SupervisionWarnings: images is deprecated: `DetectionDataset.images` property is deprecated and will be removed in `supervision-0.26.0`. Iterate with `for path, image, annotation in dataset:` instead.



0: 480x640 1 undamaged building, 4 damaged buildings, 207.2ms
Speed: 10.2ms preprocess, 207.2ms inference, 0.7ms postprocess per image at shape (1, 3, 480, 640)
Building 1: Latitude: 29.94202514838578, Longitude: -85.40205054652843
Building 2: Latitude: 29.94230109279127, Longitude: -85.40187720757848
Building 3: Latitude: 29.941941409416348, Longitude: -85.40178733740878
Building 4: Latitude: 29.942145268910508, Longitude: -85.40223697802199
Building 5: Latitude: 29.941818505480352, Longitude: -85.40159451678863
Processing image: ../data/images/test/15675.jpg


SupervisionWarnings: images is deprecated: `DetectionDataset.images` property is deprecated and will be removed in `supervision-0.26.0`. Iterate with `for path, image, annotation in dataset:` instead.


...


0: 480x640 1 undamaged building, 3 damaged buildings, 367.6ms
Speed: 8.7ms preprocess, 367.6ms inference, 0.8ms postprocess per image at shape (1, 3, 480, 640)
Building 1: Latitude: 29.938447824730822, Longitude: -85.40051069411993
Building 2: Latitude: 29.938623270297718, Longitude: -85.40030186746989
Building 3: Latitude: 29.938134844182375, Longitude: -85.39998603165489
Building 4: Latitude: 29.938134808814922, Longitude: -85.3999873088934
Processing image: ../data/images/test/15725.jpg


SupervisionWarnings: images is deprecated: `DetectionDataset.images` property is deprecated and will be removed in `supervision-0.26.0`. Iterate with `for path, image, annotation in dataset:` instead.



0: 480x640 (no detections), 173.3ms
Speed: 3.9ms preprocess, 173.3ms inference, 0.4ms postprocess per image at shape (1, 3, 480, 640)
Processing image: ../data/images/test/15724.jpg


SupervisionWarnings: images is deprecated: `DetectionDataset.images` property is deprecated and will be removed in `supervision-0.26.0`. Iterate with `for path, image, annotation in dataset:` instead.



0: 480x640 (no detections), 189.4ms
Speed: 4.0ms preprocess, 189.4ms inference, 0.5ms postprocess per image at shape (1, 3, 480, 640)
